In [1]:
pip install opencv-python==3.4.18.65

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np

In [ ]:
# init part
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
# detector_params = cv2.SimpleBlobDetector_Params()
# detector_params.filterByArea = True
# detector_params.maxArea = 1500
# detector = cv2.SimpleBlobDetector_create(detector_params)

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

#***PARAMETERS***#

# Change thresholds (0 to 255) 
params.minThreshold = 0;
params.maxThreshold = 255;

#Area - area in terms of pixels
params.filterByArea = False
params.minArea = 1000
      
#Circularity - measures how close the blob to a circle
params.filterByCircularity = True
params.minCircularity = 0.95 #"cicularity" of a square = 0.785 , "cicularity" of a circle = 1.000

#Color - 0 (select darker) to  255 (select lighter)
params.filterByColor = True
params.minCircularity = 50 

#Convexity - (Area of the Blob / Area of it’s convex hull)
params.filterByConvexity = False
params.minConvexity = 0.87

#Inertia - how elongated a shape is, 1 = circle, 0 = line
params.filterByInertia = False
params.minInertiaRatio = 0.95


params.minRepeatability=1



# def blob_process(img, threshold, detector):
#     gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
#     img = cv2.erode(img, None, iterations=2)
#     img = cv2.dilate(img, None, iterations=4)
#     img = cv2.medianBlur(img, 5)
#     keypoints = detector.detect(img)
#     print(keypoints)
#     return keypoints


def nothing(x):
    pass


def main():
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('image')
    cv2.createTrackbar('threshold', 'image', 20, 100, nothing)
    while True:
        _, frame = cap.read()
        threshold = r = cv2.getTrackbarPos('threshold', 'image')
        if (threshold < 20):
            threshold = 20
        #print("Threshold Raw")
        #print(threshold)
        threshold = threshold/100
        #print("Threshold Percent")
        #print(threshold)

        # Create a detector with the parameters

        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
          detector = cv2.SimpleBlobDetector(params)
        else : 
          detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(frame)
        
        #print("Keypoints")
        #print(keypoints)

        
        eye = cv2.drawKeypoints(frame, keypoints, frame, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        #print("Eye")
        print(eye)

        cv2.imshow('image', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()